# ATACseq pipeline Mouse

This jupyter-notebook automates the ATACseq pre-processing pipeline as well as the basic analysis

In the following section we assume that:
- The reference genome has to be already indexed with bowtie
- The tutorials are introduced with the example file mentioned in the tutorial

### Data Description

In this pipeline we used data from GEO series GSE65360. This dataset is baed on the paper Buenrostro JD, Wu B, Litzenburger UM, Ruff D et al. Single-cell chromatin accessibility reveals principles of regulatory variation. Nature 2015 Jul 23;523(7561):486-90. PMID: [26083756](https://www.ncbi.nlm.nih.gov/pubmed/26083756)

They profiled single cell epigenomes, scATAC-seq assay, across 8 cell types and 4 targeted cell manipulations. The complete data set contains a total of 1,632 assayed wells.

For their experiment they used four different platforms
- Illumina MiSeq (Homo sapiens)
- Illumina HiSeq 2500 (Homo sapiens)
- Illumina NextSeq 500 (Homo sapiens)
- Illumina NextSeq 500 (Mus musculus)

The fastq files have varying length of sequencing reads. An overview of different cell types they used are as follows

__Human__
- singles-H1ESC: The H1 human embryonic stem cells were obtained from WiCell and grown in chemically defined mTeSR1 medium (STEMCELL Technologies) on Matrigel-coated plates.
- singles-GM: GM12878 lymphoblastoid cells (a kind gift from the Snyder lab) were grown in RPMI 1640 with 2 mmol/L L-glutamine, 15% FBS and 1% Pen/Strep
- singles-GM-TNFa6h: GM12878 lymphoblastoid cells (a kind gift from the Snyder lab) were grown in RPMI 1640 with 2 mmol/L L-glutamine, 15% FBS and 1% Pen/Strep. GM12878 cells were treated with 25 ng/mL rhTNFα (eBiosciences) for 6 h
- singles-K562: K562 (ATCC) chronic myeloid leukemia cells were maintained in Iscove’s modified Dulbecco’s medium (IMDM) containing 10% FBS (HyClone, Thermo Scientific) and 1% Penicillin Streptomycin (Pen/Strep).
- singles-K562-CDKi: K562 (ATCC) chronic myeloid leukemia cells were maintained in Iscove’s modified Dulbecco’s medium (IMDM) containing 10% FBS (HyClone, Thermo Scientific) and 1% Penicillin Streptomycin (Pen/Strep). K562 cells were treated with 1 µM CDK4/6 inhibitor (PD 0332991, Pfizer) for 24 h. [Acute myeloid leukemia (AML), also known as acute myelogenous leukemia or acute nonlymphocytic leukemia (ANLL), is a cancer of the myeloid line of blood cells, characterized by the rapid growth of abnormal white blood cells that accumulate in the bone marrow and interfere with the production of normal blood cells. AML is the most common acute leukemia affecting adults, and its incidence increases with age. Although AML is a relatively rare disease, accounting for roughly 1.2% of cancer deaths in the United States,[1] its incidence is expected to increase as the population ages]
- singles-K562-Imat1hr: K562 (ATCC) chronic myeloid leukemia cells were maintained in Iscove’s modified Dulbecco’s medium (IMDM) containing 10% FBS (HyClone, Thermo Scientific) and 1% Penicillin Streptomycin (Pen/Strep). K562 cells were treated with 1 µM Imatinib (Gleevec, Novartis) for 1 h.
- singles-K562-JNKi: K562 (ATCC) chronic myeloid leukemia cells were maintained in Iscove’s modified Dulbecco’s medium (IMDM) containing 10% FBS (HyClone, Thermo Scientific) and 1% Penicillin Streptomycin (Pen/Strep). K562 cells were treated with 10 µM JNK inhibitor VIII (CAS 894804-07-0, Calbiochem) for 24 h
- singles-TF1: The erythroleukemia cell line TF-1 (kind gift from the Majeti lab, Stanford) was maintained in RPMI 1640 with 10% FBS, 1% Pen/Strep and 2 ng/mL rhGM-CSF (Peprotech).[Acute erythroid leukemia or Di Guglielmo syndrome is a rare form of acute myeloid leukemia (less than 5% of AML cases[1]) where the myeloproliferation is of erythroblastic precursors. It is defined as type "M6"]
- singles-HL60: The promyelocytic leukemia cells HL-60 were grown in IMDM containing 20% FBS and 1% Pen/Strep. [Acute promyelocytic leukemia (APML, APL) is the M3 subtype of acute myelogenous leukemia (AML), a cancer of the white blood cells.[1] In APL, there is an abnormal accumulation of immature granulocytes called promyelocytes. The disease is characterized by a chromosomal translocation involving the retinoic acid receptor alpha (RARα or RARA) gene and is distinguished from other forms of AML by its responsiveness to all-trans retinoic acid (ATRA; also known as tretinoin) therapy]
- singles-BJ: Human BJ fibroblasts (ATCC) were maintained in Eagle’s minimum essential medium (EMEM, ATCC) supplemented with 10% FBS and 1% Pen/Strep.

__Mouse__
- singles-EML: The mouse EML hematopoietic cells were grown in IMDM containing 20% horse serum, 2 mM L-Glutamine, 1% Pen/Strep and 100 ng/mL recombinant SCF (Peprotech). [Hematopoietic stem cells (HSCs) or hemocytoblasts are the stem cells that give rise to all the other blood cells through the process of haematopoiesis]
- singles-mESC: V6.5 mouse embryonic stem cells were grown in Glasgow Minimum Essential Medium (GMEM) supplemented with 10% FBS, 2 mM glutamine, 1% Pen/Strep, 1 mM sodium pyruvate, 1000 units/ml Leukemia Inhibitory Factor (LIF, Millipore), 1X Minimum Essential Medium NonEssential Amino Acids (MEM NEAA, Invitrogen) and 50 uM β-Mercaptoethanol. Cells were maintained on gelatin-coated dishes without feeders


### Data processing summary

We used `trimmomatic` to trim the adapter sequences. Paired-end reads were aligned to `hg19` or `mm10` using `BOWTIE2`. We used the `-X2000` parameter in `BOWTIE2` allowing fragments of up to 2 Kb to align. This considers that the read pairs within 2000 bp are concordant mapping. Duplicates were removed using `PICARD` tools `Markduplicate`. Reads are then filtered for alignment quality of >Q30 and were required to be properly paired. Reads mapping to the mitochondria, unmapped contigs and chromosome Y are not considered.

We used MACS2 to call ATAC-seq peaks. The parameters that we used for MACS2 are (--nomodel --nolambda --keep-dup all --call-summits). Peaks were filtered using the consensus excludable [ENCODE blacklist](http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeMapability/). We have saved it here in our local disk. Using the filtered peak set, peak summits were extended +/-250 bps.

## Explanation of different parameters for the tools

For convenience of the user an explanation of the parameters used with different tools are given below

### Bowtie options
- Pass the trimmed fastq files to bowtie2 for mapping.
- Bowtie2 mapping with default parameters using 16 cores, except pass a -X 2000, 
- which treat read pairs that are within 2000 bp as concordant mapping.
- This only affects concordant rate, it won't affect mapping rate.
- -X 2000 is suggested by the Greenleaf paper. That's the reason I used this.

### Samtools option
- Convert the sam file, including sam header, to bam file, and remove non-mapped reads.
- The following command will do these in one go.
- -S option indicates the input is a sam file
- -b option asked samtools to write output as a bam file
- -h option asked samtools to print header, it is very important to include this "-h"
- when converting between sam <-> bam files.
- -F 4 option removes non-mapped reads
- -f2 only include reads with all bits set in INT set in FLAG [0]
- -q only include reads with mapping quality >= INT [0]

### Picard tools
- Picard tools is used to remove the duplicates

### Removing the Blacklists
Functional genomics experiments based on next-gen sequencing (e.g. ChIP-seq, MNase-seq, DNase-seq, FAIRE-seq) that measure biochemical activity of various elements in the genome often produce artifact signal in certain regions of the genome. It is important to keep track of and filter artifact regions that tend to show artificially high signal (excessive unstructured anomalous reads mapping). Below is a list of comprehensive empirical blacklists identified by the ENCODE and modENCODE consortia. Note that these blacklists were empirically derived from large compendia of data using a combination of automated heuristics and manual curation. These blacklists are applicable to functional genomic data based on short-read sequencing (20-100bp reads). These are not directly applicable to RNA-seq or any other transcriptome data types. The blacklisted regions typically appear u niquely mappable so simple mappability filters do not remove them. These regions are often found at specific types of repeats such as centromeres, telomeres and satellite repeats. It is especially important to remove these regions that computing measures of similarity such as Pearson correlation between genome-wide tracks that are especially affected by outliers.

download the `hg19 blacklist` into the data folder by `wget` [hg19 Blacklist](http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeMapability/wgEncodeDacMapabilityConsensusExcludable.bed.gz)

### Perform peak calling using macs2 (2 2.1.0.20151222)
Perform peak calling and generate bedGraph using 50 bp reads centred on 5 prime cutting end, within a 50-bp smoothing window for bedGraph signal track. The parameters for macs2 are as follows

- `-t/--treatment` FILENAME: This is the only REQUIRED parameter for MACS. File can be in any supported format specified by –format option. Check –format for detail. If you have more than one alignment files, you can specify them as `-t A B C`. MACS will pool up all these files together.
- `-n/--name`: The name string of the experiment. MACS will use this string NAME to create output files like ‘NAME_peaks.xls’, ‘NAME_negative_peaks.xls’, ‘NAME_peaks.bed’ , ‘NAME_summits.bed’, ‘NAME_model.r’ and so on. So please avoid any confliction between these filenames and your existing files.
- `-g/--gsize` <br><br> PLEASE assign this parameter to fit your needs! <br> <br> It’s the mappable genome size or effective genome size which is defined as the genome size which can be sequenced. Because of the repetitive features on the chromsomes, the actual mappable genome size will be smaller than the original size, about 90% or 70% of the genome size. The default hs – 2.7e9 is recommended for UCSC human hg18 assembly. Here are all precompiled parameters for effective genome size: <br> __hs__:	2.7e9 <br> __mm__:	1.87e9 <br> __ce__:	9e7 <br> __dm__: 1.2e8
- `-f/--format FORMAT`: Format of tag file, can be “ELAND”, “BED”, “ELANDMULTI”, “ELANDEXPORT”, “ELANDMULTIPET” (for pair-end tags), “SAM”, “BAM”, “BOWTIE” or “BAMPE”. Default is “AUTO” which will allow MACS to decide the format automatically. “AUTO” is also usefule when you combine different formats of files.
- `--nomodel`: While on, MACS will bypass building the shifting model.
- `--shift` : <br> Note, this is NOT the legacy –shiftsize option which is replaced by –extsize! You can set an arbitrary shift in bp here. Please Use discretion while setting it other than default value (0). When –nomodel is set, MACS will use this value to move cutting ends (5’) then apply –extsize from 5’ to 3’ direction to extend them to fragments. When this value is negative, ends will be moved toward 3’->5’ direction, otherwise 5’->3’ direction. Recommended to keep it as default 0 for ChIP-Seq datasets, or -1 * half of EXTSIZE together with –extsize option for detecting enriched cutting loci such as certain DNAseI-Seq datasets. Note, you can’t set values other than 0 if format is BAMPE for paired-end data. Default is 0. <br><br> Here are some examples for combining –shift and –extsize: <br><br>1. To find enriched cutting sites such as some DNAse-Seq datasets. In this case, all 5’ ends of sequenced reads should be extended in both direction to smooth the pileup signals. If the wanted smoothing window is 200bps, then use ‘–nomodel –shift -100 –extsize 200’.<br><br>2. For certain nucleosome-seq data, we need to pileup the centers of nucleosomes using a half-nucleosome size for wavelet analysis (e.g. NPS algorithm). Since the DNA wrapped on nucleosome is about 147bps, this option can be used: ‘–nomodel –shift 37 –extsize 73’.
- `--extsize`: While ‘–nomodel’ is set, MACS uses this parameter to extend reads in 5’->3’ direction to fix-sized fragments. For example, if the size of binding region for your transcription factor is 200 bp, and you want to bypass the model building by MACS, this parameter can be set as 200. This option is only valid when –nomodel is set or when MACS fails to build model and –fix-bimodal is on.
- `-B/--bdg`: If this flag is on, MACS will store the fragment pileup, control lambda, -log10(pvalue) and -log10(qvalue) scores in bedGraph files. The bedGraph files will be stored in current directory named NAME+’_treat_pileup.bdg’ for treatment data, NAME+’_control_lambda.bdg’ for local lambda values from control, NAME+’_treat_pvalue.bdg’ for Poisson pvalue scores (in -log10(pvalue) form), and NAME+’_treat_qvalue.bdg’ for q-value scores from Benjamini–Hochberg–Yekutieli procedure [False discovery rate](http://en.wikipedia.org/wiki/False_discovery_rate#Dependent_tests).
- `--SPMR`: Normalises the coverage plot values by millions of reads. 
- `--call-summits`: MACS will now reanalyze the shape of signal profile (p or q-score depending on cutoff setting) to deconvolve subpeaks within each peak called from general procedure. It’s highly recommended to detect adjacent binding events. While used, the output subpeaks of a big peak region will have the same peak boundaries, and different scores and peak summit positions.

### macs2 _Output Files_ Format
1. NAME_peaks.xls is a tabular file which contains information about called peaks. You can open it in excel and sort/filter using excel functions. Information include:
    * chromosome name
    * start position of peak
    * end position of peak
    * length of peak region
    * absolute peak summit position
    * pileup height at peak summit, -log10(pvalue) for the peak summit (e.g. pvalue =1e-10, then this value should be 10)
    * fold enrichment for this peak summit against random Poisson distribution with local lambda, -log10(qvalue) at peak summit
2. NAME_peaks.narrowPeak is BED6+4 format file which contains the peak locations together with peak summit, pvalue and qvalue. You can load it to UCSC genome browser. Definition of some specific columns are:
    * 5th: integer score for display
    * 7th: fold-change
    * 8th: -log10(pvalue)
    * 9th: -log10(qvalue)
    * 10th: relative summit position to peak start
    The file can be loaded directly to UCSC genome browser. Remove the beginning track line if you want to analyze it by other tools.



### Convert bedGraph to bigWig
- To call peaks and generate bedGraph using a different smoothing window, change --shift 0.5*window --extsize window
- For example, --shift 75 --extsize 150 will use 150 bp reads centred on 5 prime cutting end to call peaks, and use a 150-bp smoothing window to generate bedGraph


## Importing python modules

In [1]:
import subprocess, os, sys, signal, pip
import multiprocessing
import threading
import time
import numpy as np

## Setting the folders and tools

First we configure the __Folders__

In [2]:
# Setting the output folder where all the results would be stored
outputFolder = '/home/baker/my-dpsf-scratch-share/Buenrostro_pipeline/Mouse_data_analysis/output/'

# inputFolder : Folder name with all the fastq files
intputFolder = '/home/baker/my-single-cell-share/Buenrostro_ATAC-seq_data/Mouse_fastq_files'

# Blacklisted sequences
blackListFile = '/home/baker/my-hydra-share/Packages/Blacklist_regions/consensusBlacklist.bed'

# Setting up the genome folder
#ref_genome = '/home/baker/my-mm10-index-share/Mus_musculus.GRCm38.71.fa'
ref_genome = '/home/baker/my-genomeIndex-share/mm10_random_chrM_chrUn/bowtie2/mm10_random_chrM_chrUn'

### Setting up the softwares

We configure the __software__ parameters here. If the required softwares are not in the PATH you can manually set their location here

In [3]:
bowtie_path = '/home/baker/Downloads/bowtie2-2.2.6/bowtie2'
samtools_path = 'samtools'
macs2_path = 'macs2'
bdg2bw_path = '/home/baker/my-hydra-share/Packages/bdg2bw'
picard_path = '/home/baker/picard/dist/picard.jar'
intersectBed_path = 'intersectBed'
MarkDuplicate_path = '/home/baker/my-hydra-share/Packages/picard-tools-1.115/MarkDuplicates.jar'
Trimmomatic_path = '/home/baker/my-hydra-share/Packages/Trimmomatic-0.35/trimmomatic-0.35.jar'
fastqc_path ='/home/baker/my-hydra-share/Packages/FastQC/fastqc'
Trimmomatic_Adapter = '/home/baker/my-hydra-share/Packages/Trimmomatic-0.35/adapters/NexteraPE-PE.fa:2:30:10:1:true TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:25'

In [4]:
softwares = {    
    'bowtie2': bowtie_path,
    'macs2': macs2_path,
    'bdg2bw': bdg2bw_path,
    'samtools': samtools_path,
    'picard': picard_path,
    'intersectBed': intersectBed_path,
    'MarkDuplicate': MarkDuplicate_path,
    'trimmomatic': Trimmomatic_path,
    'fastqc': fastqc_path,
    'Trimmomatic_Adapter': Trimmomatic_Adapter
}

### Creating the output folders
We now create the output folders where all the processed files will be stored

In [5]:
output_folders = [ 'Bowtie_files', 'Blacklist_removed' # Trimmo, Bowtie, Blacklist Removed Files
                 , 'Trimmomatic_Files', 'Fastqc_SE_Files' 
                 , 'Duplicates_removed', 'Macs2_files'              # Synced R1 and R2
                 , 'Merged_BAM', 'Merged_Macs2_files'
                 , 'Bam_Files_Filtered_on_Library_Size', 'Merged_Filtered_BAM'
                 ,  'Merged_Filtered_Macs2_files', 'Filtered_Macs2_files'
                 ]

In [6]:
for folder in output_folders:
    os.makedirs(os.path.join(outputFolder, folder))

### Custom functions

Here we define __custom__ functions that we will use for file processing

In [7]:
remove_extension = lambda x: x.split('.')[0]

The following functions deals with the inputs and the outputs

In [8]:
def get_args(read1, read2, ref_genome, blackListFile, output_folders):
    '''Set the input and output path for a given pair of reads'''
    r1_shortname = remove_extension(os.path.basename(read1))
    #sample_name = r1_shortname.split('_')[0] + '_'+ r1_shortname.split('_')[1]
    sample_name = r1_shortname.split('_')[0]

    args = {  
        'r1_input': read1,
        'r2_input': read2,
        'ref_genome': ref_genome,
        'blackListFile': blackListFile,
        'sample_name': sample_name,
    }
    
    # output_paths = {folder: os.path.join(outputFolder, folder, r1_shortname) for folder in output_folders}
    output_paths = {folder: os.path.join(outputFolder, folder, sample_name) for folder in output_folders}
    
    return dict(args, **output_paths)

## Running the pipeline

In order to run the pipeline we first define the commands. A brief explanation of the tools are given below

- `trimmomatic`: To trim the adapter and low quality reads
- `bowti2`: To map the reads with genome. Here we are mapping with Human genome. The genome is defined accordingly in the `{ref_genome}` folder
- `samtools`: In this samtools view command we retain only the properly paired reads and reads with quality above 30
- `intersectBed`: We remove the blacklists
- `MarkDuplicate`: With this picard tool we are removing the duplicates [where duplicate reads are defined as originating from a single fragment of DNA which can arise during sample preparation e.g. library construction using PCR]. MarkDuplicate just marks the duplicate sequences but to make sure that it removes the duplcate from the bam file the parameter `REMOVE_DUPLICATES` is set to `TRUE`
- `samtools idxstats`: samtools idxstas are used to generate the mapping quality of the reads
- `macs2 callpeak`: Here we call the peaks

In [9]:
cmds = [
    
    #'java -jar {trimmomatic} PE -phred33 {r1_input} {r2_input} {Trimmomatic_Files}_r1_paired.fq {Trimmomatic_Files}_r1_unpaired.fq {Trimmomatic_Files}_r2_paired.fq {Trimmomatic_Files}_r2_upaired.fq ILLUMINACLIP:{Trimmomatic_Adapter}',
    
    #'{bowtie2} -p 12 -X 2000 --dovetail -x {ref_genome} -1 {Trimmomatic_Files}_r1_paired.fq -2 {Trimmomatic_Files}_r2_paired.fq -S {Bowtie_files}.sam 2> {Bowtie_files}_allignment.log',
    
    '{bowtie2} -p 12 -X 2000 --dovetail -x {ref_genome} -1 {r1_input} -2 {r2_input} -S {Bowtie_files}.sam 2> {Bowtie_files}_allignment.log',
    
    '{samtools} view -SbhF 4 -f2 -q30 {Bowtie_files}.sam > {Bowtie_files}.bam' ,
    
    'intersectBed -v -abam {Bowtie_files}.bam -b {blackListFile} > {Blacklist_removed}_noexclu.bam ' ,
    
    '{samtools} sort {Blacklist_removed}_noexclu.bam {Blacklist_removed}_noexclu_sorted' ,
    
    'java -Xmx2g -jar {MarkDuplicate}  INPUT={Blacklist_removed}_noexclu_sorted.bam OUTPUT={Duplicates_removed}_nodup.bam METRICS_FILE={Duplicates_removed}_nodup_stats REMOVE_DUPLICATES=True' ,
    
    '{samtools} sort {Duplicates_removed}_nodup.bam {Duplicates_removed}_nodup_sorted' ,
    
    '{samtools} index {Duplicates_removed}_nodup_sorted.bam' ,
    
    '{samtools} idxstats {Duplicates_removed}_nodup_sorted.bam > {Duplicates_removed}_nodup_sorted_chrom_stat.txt' ,
    
    'macs2 callpeak -t {Duplicates_removed}_nodup_sorted.bam -n {Macs2_files} -q 0.01 -g hs -f BAM --nomodel --nolambda --shift -75 --extsize 150 -B --SPMR --keep-dup all --call-summits',
        
    '{bdg2bw} {Macs2_files}_treat_pileup.bdg /home/baker/my-hydra-share/Packages/Homer_4.6/bin/hg19.chrom.sizes',
        
]

Get the reads from the `inputFolder`

In [10]:
for root, folders, files in os.walk(intputFolder):
    files = [f for f in files if not f.startswith('.')] #remove hidden files if there exist
    reads1 = sorted([os.path.join(root, f) for f in files if '_1' in f])
    reads2 = sorted([os.path.join(root, f) for f in files if '_2' in f])

Print the read names. This is to make sure that the correct files are picked up. This can be run just once as it is expected to produce large output. 

In [11]:
print (reads1, reads2)

['/home/baker/my-single-cell-share/Buenrostro_ATAC-seq_data/Mouse_fastq_files/MusmusculusmESC-well-10_1.fastq', '/home/baker/my-single-cell-share/Buenrostro_ATAC-seq_data/Mouse_fastq_files/MusmusculusmESC-well-11_1.fastq', '/home/baker/my-single-cell-share/Buenrostro_ATAC-seq_data/Mouse_fastq_files/MusmusculusmESC-well-12_1.fastq', '/home/baker/my-single-cell-share/Buenrostro_ATAC-seq_data/Mouse_fastq_files/MusmusculusmESC-well-13_1.fastq', '/home/baker/my-single-cell-share/Buenrostro_ATAC-seq_data/Mouse_fastq_files/MusmusculusmESC-well-14_1.fastq', '/home/baker/my-single-cell-share/Buenrostro_ATAC-seq_data/Mouse_fastq_files/MusmusculusmESC-well-15_1.fastq', '/home/baker/my-single-cell-share/Buenrostro_ATAC-seq_data/Mouse_fastq_files/MusmusculusmESC-well-16_1.fastq', '/home/baker/my-single-cell-share/Buenrostro_ATAC-seq_data/Mouse_fastq_files/MusmusculusmESC-well-17_1.fastq', '/home/baker/my-single-cell-share/Buenrostro_ATAC-seq_data/Mouse_fastq_files/MusmusculusmESC-well-18_1.fastq', 

The platform is now set to run the commands. Below we run the command sequentially. However, it is always good to first check that the right commands are executed. To test that one can uncomment the print command

In [12]:
def run_cmd(cmds, args):
    for cmd in cmds: 
        #print(cmd.format(**args))
        subprocess.call(cmd.format(**args), shell=True)

Looking at the number of cores in the node

In [13]:
print("Total Cores:",multiprocessing.cpu_count())

Total Cores: 8


### Sequential run

In [14]:
for read1, read2 in zip(reads1, reads2):
    args = get_args(read1, read2, ref_genome, blackListFile, output_folders)
    args = dict(args, **softwares)
    run_cmd(cmds,args)